In [255]:
import pandas as pd
df = pd.read_csv("USDJPY2.csv")

In [256]:
#df = df.query("Date>='1997-01-02'")
#df = df.sort_values('日付け', ascending=True).reset_index()
df

,日付け,終値,始値,高値,安値,前日比%
0,2021年04月29日,108.91,108.60,109.23,108.43,0.29%
1,2021年04月28日,108.59,108.74,109.08,108.57,-0.08%
2,2021年04月27日,108.68,108.07,108.78,108.07,0.56%
3,2021年04月26日,108.08,107.99,108.21,107.64,0.16%
4,2021年04月23日,107.91,107.96,108.15,107.47,-0.05%
...,...,...,...,...,...,...
4254,2005年01月07日,104.80,104.97,105.17,103.83,-0.26%
4255,2005年01月06日,105.07,104.12,105.20,103.85,0.94%
4256,2005年01月05日,104.09,104.56,105.03,103.73,-0.50%
4257,2005年01月04日,104.61,102.77,104.81,102.41,1.84%


In [257]:
import talib as ta
import numpy as np

close = np.array(df["終値"])

#特徴量を入れるための空のdataframeを作成
df_feature = pd.DataFrame(index=range(len(df)),columns=["SMA5/current", "SMA20/current","RSI","MACD","BBANDS+2σ","BBANDS-2σ"])

#以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる

#単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
df_feature["SMA5/current"]= ta.SMA(close, timeperiod=5) / close
df_feature["SMA20/current"]= ta.SMA(close, timeperiod=20) / close

#RSI
df_feature["RSI"] = ta.RSI(close, timeperiod=14)

#MACD
df_feature["MACD"], _ , _= ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

#ボリンジャーバンド 
upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=3, nbdevdn=3)
df_feature["BBANDS+2σ"] = upper / close
df_feature["BBANDS-2σ"] = lower / close

In [258]:
def classify(x):
#前日比が-0.2%以下ならグループ０
    if x <= -0.2:
        return 0
#前日比が0.2%<x<0.2%ならグループ1
    elif -0.2 < x < 0.2:
        return 1
#前日比が0.2%以上ならグループ２
    elif 0.2 <= x:
        return 2

In [259]:
df["前日比_float"] = df["前日比%"].apply(lambda x: float(x.replace("%", "")))

#前日比%の分類の仕方。できるだけ各クラスのサンプルが等しいようにわける
def classify(x):
    if x <= -0.2:
        return 0
    elif -0.2 < x < 0.2:
        return 1
    elif 0.2 <= x:
        return 2


df["前日比_classified"] = df["前日比_float"].apply(lambda x: classify(x))

#教師にしたいデータを一日ずつずらす
#df_y = df["前日比_classified"].shift()
df_y = df["前日比_classified"].shift()

In [260]:
df["前日比_float"].head(20)

0     0.29
1    -0.08
2     0.56
3     0.16
4    -0.05
5    -0.08
6    -0.03
7    -0.06
8    -0.57
9     0.03
10   -0.16
11   -0.13
12   -0.29
13   -0.26
14    0.38
15   -0.55
16    0.09
17   -0.39
18   -0.49
19    0.10
Name: 前日比_float, dtype: float64

In [261]:
df["前日比_classified"].head(20)

0     2
1     1
2     2
3     1
4     1
5     1
6     1
7     1
8     0
9     1
10    1
11    1
12    0
13    0
14    2
15    0
16    1
17    0
18    0
19    1
Name: 前日比_classified, dtype: int64

In [262]:
df_y

0       NaN
1       2.0
2       1.0
3       2.0
4       1.0
       ... 
4254    0.0
4255    0.0
4256    2.0
4257    0.0
4258    2.0
Name: 前日比_classified, Length: 4259, dtype: float64

In [263]:
df["前日比_classified"].shift()

0       NaN
1       2.0
2       1.0
3       2.0
4       1.0
       ... 
4254    0.0
4255    0.0
4256    2.0
4257    0.0
4258    2.0
Name: 前日比_classified, Length: 4259, dtype: float64

In [264]:
df_feature

,SMA5/current,SMA20/current,RSI,MACD,BBANDS+2σ,BBANDS-2σ
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,1.004856,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4254,0.987023,0.984823,59.170051,-0.280442,1.007137,0.962510
4255,0.989550,0.982583,60.854391,-0.121720,1.006498,0.958668
4256,1.002075,0.992041,52.404241,-0.074155,1.016639,0.967442
4257,0.999465,0.987535,55.903220,0.005438,1.013294,0.961775


In [265]:
df["前日比_float"].tail()

4254   -0.26
4255    0.94
4256   -0.50
4257    1.84
4258    0.24
Name: 前日比_float, dtype: float64

In [266]:
df_xy = pd.concat([df_feature, df_y], axis=1)
df_xy = df_xy.dropna(how="any")

In [267]:
pd.concat([df_feature, df_y,df["終値"]], axis=1).tail(30)

,SMA5/current,SMA20/current,RSI,MACD,BBANDS+2σ,BBANDS-2σ,前日比_classified,終値
4229,0.995554,0.992593,53.500983,-0.096395,1.007962,0.977224,0.0,105.71
4230,0.994861,0.992135,54.505028,-0.033640,1.007354,0.976916,1.0,105.85
4231,0.997293,0.994898,52.667933,-0.000843,1.008980,0.980816,2.0,105.64
4232,0.998657,0.994185,53.708113,0.035233,1.008562,0.979809,1.0,105.77
4233,1.006656,1.002551,46.147079,-0.008698,1.017150,0.987952,2.0,104.87
4234,1.011009,1.009664,40.848598,-0.104443,1.025579,0.993749,2.0,104.10
4235,1.004863,1.005801,44.081265,-0.149549,1.022125,0.989478,0.0,104.46
4236,1.008798,1.013052,38.982783,-0.247001,1.031653,0.994451,2.0,103.66
4237,1.004261,1.012119,39.547521,-0.315752,1.032137,0.992101,1.0,103.72
4238,1.002431,1.012168,39.221740,-0.370006,1.033716,0.990621,1.0,103.67


In [268]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import optuna

X_train, X_test, Y_train, Y_test = train_test_split(df_xy[["SMA5/current", "SMA20/current","RSI","MACD","BBANDS+2σ","BBANDS-2σ"]],df_xy["前日比_classified"], train_size=0.8)

def objective(trial):
    min_samples_split = trial.suggest_int("min_samples_split", 2,16)
    max_leaf_nodes = int(trial.suggest_discrete_uniform("max_leaf_nodes", 4,64,4))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    n_estimators = int(trial.suggest_discrete_uniform("n_estimators", 50,500,50))
    max_depth = trial.suggest_int("max_depth", 3,10)
    clf = RandomForestClassifier(random_state=1, n_estimators = n_estimators, max_leaf_nodes = max_leaf_nodes, max_depth=max_depth, max_features=None,criterion=criterion,min_samples_split=min_samples_split)
    clf.fit(X_train, Y_train)
    return 1 - accuracy_score(Y_test, clf.predict(X_test))

study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=100)
study.optimize(objective, n_trials=3)


[I 2021-05-04 22:42:03,425] A new study created in memory with name: no-name-27dcd37d-99f9-4483-be4d-288e869ad939
[I 2021-05-04 22:42:16,046] Trial 0 finished with value: 0.38179669030732866 and parameters: {'min_samples_split': 10, 'max_leaf_nodes': 48.0, 'criterion': 'gini', 'n_estimators': 500.0, 'max_depth': 10}. Best is trial 0 with value: 0.38179669030732866.
[I 2021-05-04 22:42:27,223] Trial 1 finished with value: 0.3806146572104019 and parameters: {'min_samples_split': 14, 'max_leaf_nodes': 60.0, 'criterion': 'gini', 'n_estimators': 450.0, 'max_depth': 8}. Best is trial 0 with value: 0.38179669030732866.
[I 2021-05-04 22:42:36,146] Trial 2 finished with value: 0.39125295508274227 and parameters: {'min_samples_split': 2, 'max_leaf_nodes': 56.0, 'criterion': 'gini', 'n_estimators': 500.0, 'max_depth': 4}. Best is trial 2 with value: 0.39125295508274227.


In [269]:
#ベストな結果、パラメータの表示
print('Accuracy: {}'.format(1-study.best_value))
print("Best hyperparameters: {}".format(study.best_trial.params))

Accuracy: 0.6087470449172577
Best hyperparameters: {'min_samples_split': 2, 'max_leaf_nodes': 56.0, 'criterion': 'gini', 'n_estimators': 500.0, 'max_depth': 4}


In [270]:
# チューニングしたハイパーパラメーターをフィット
optimised_rf = RandomForestClassifier(random_state=1, n_estimators = int(study.best_params['n_estimators']), max_leaf_nodes = int(study.best_params['max_leaf_nodes']), max_depth=study.best_params['max_depth'], max_features=None,criterion=study.best_params['criterion'],min_samples_split=study.best_params['min_samples_split'])

In [271]:
optimised_rf.fit(X_train, Y_train)
Y_pred = optimised_rf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.6087470449172577
